In [1]:
import numpy as np
# np.set_printoptions(precision=3)

def generate_non_singular_matrix(n):
    while True:
        A = np.random.rand(n, n) + 1j * np.random.rand(n, n)
        if np.linalg.det(A) != 0:
            return A

In [2]:
def QR_decomposition(A):
    n = A.shape[0]
    Q = np.zeros((n, n))
    R = np.zeros((n, n))

    for k in range(n):
        Q[:, k] = A[:, k]
        for i in range(k):
            R[i, k] = np.dot(Q[:, i], A[:, k])
            Q[:, k] -= R[i, k] * Q[:, i]
        Q[:, k] /= np.linalg.norm(Q[:, k])
        R[k, k] = np.dot(np.conj(Q[:, k]), A[:, k])

    return Q, R

In [3]:
def permutation(v):
    """
    Constructs a permutation matrix from a complex vector following the order of the modulus of the entries.
    Parameters:
        v (ndarray): Complex vector.
    Returns:
        P (ndarray): Permutation matrix.
    """
    # Sort indices of v based on the modulus of entries
    sorted_indices = np.argsort(abs(v))[::-1]

    # Create permutation matrix
    n = len(v)
    P = np.zeros((n, n))
    for i, idx in enumerate(sorted_indices):
      P[i, idx] = 1

    return P

In [4]:
def qr_method(A, tol, x):
    """
    QR method to compute eigenvalues.

    Parameters:
        A (ndarray): Matrix A.
        tol (float): Tolerance for loop exit (default=1e-6).
        x (ndarray): Initial guess (optional).
    Returns:
        Ak (ndarray): Upper triangular matrix with diagonal values sorted by their norm.
        V (ndarray): Invertible matrix such that A = V Ak V^*.
    """
    error = tol + 1
    Ak = np.copy(A)
    V = np.eye(A.shape[0])

    while error > tol:
        Atmp = np.copy(Ak)

        epsilon = np.random.randn()
        Qk, Rk = QR_decomposition(Ak + epsilon * np.eye(Ak.shape[0]))
        Ak = Rk @ Qk - epsilon * np.eye(Ak.shape[0])

        P = permutation(np.diag(Ak))
        Ak = np.linalg.inv(P) @ Ak @ P
        V = V @ Qk @ P

        error = np.linalg.norm(np.diag(Ak) - np.diag(Atmp))

    return Ak, V

In [5]:
tol=1e-20

In [6]:
# Test on a random 4x4 Hermitian matrix A
np.random.seed(42)
# A = np.random.rand(4, 4) + 1j * np.random.rand(4, 4)
A = generate_non_singular_matrix(4)
A = (A + A.conj().T) / 2  # Making A Hermitian
print("Random Hermitian matrix A:")
print(A)

Random Hermitian matrix A:
[[0.37454012+0.j         0.55336647-0.04354823j 0.66655448-0.01206248j
  0.71555056-0.15059271j]
 [0.55336647+0.04354823j 0.15599452+0.j         0.38307809-0.24651566j
  0.53925763+0.15995572j]
 [0.66655448+0.01206248j 0.38307809+0.24651566j 0.02058449+0.j
  0.57586741-0.04665521j]
 [0.71555056+0.15059271j 0.53925763-0.15995572j 0.57586741+0.04665521j
  0.18340451+0.j        ]]


In [7]:
x = np.random.rand(A.shape[0])
print("Initial guess x: ", x)

Initial guess x:  [0.06505159 0.94888554 0.96563203 0.80839735]


In [8]:
# Compute Ak and V using QR method
Ak, V = qr_method(A, tol = tol, x = x)
print("\nUpper triangular matrix Ak:")
Aktmp = np.copy(Ak)
for i in range(4):
  for j in range(4):
    if abs(Aktmp[i,j]) < 1e-6:
      Aktmp[i,j] = 0
print(Aktmp)
print("\nInvertible matrix V:")
print(V)

C:\Users\Minhao\AppData\Local\Temp\ipykernel_11964\2274709163.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  Q[:, k] = A[:, k]
C:\Users\Minhao\AppData\Local\Temp\ipykernel_11964\2274709163.py:12: ComplexWarning: Casting complex values to real discards the imaginary part
  R[k, k] = np.dot(np.conj(Q[:, k]), A[:, k])
C:\Users\Minhao\AppData\Local\Temp\ipykernel_11964\2274709163.py:9: ComplexWarning: Casting complex values to real discards the imaginary part
  R[i, k] = np.dot(Q[:, i], A[:, k])



Upper triangular matrix Ak:
[[ 1.93441705  0.          0.          0.        ]
 [ 0.         -0.50977902  0.          0.        ]
 [ 0.          0.         -0.45014823  0.        ]
 [ 0.          0.          0.         -0.23996615]]

Invertible matrix V:
[[-0.58375278  0.38892398  0.63832317  0.31704   ]
 [-0.43543757 -0.20438914  0.15490977 -0.86291492]
 [-0.44681529 -0.79510036 -0.11907535  0.39241888]
 [-0.5195988   0.41806535 -0.74455863  0.02951092]]


 (ii) $VA_kV^∗ = A$

In [9]:
# Check if A = V Ak V^*
reconstructed_A = V @ Ak @ np.linalg.inv(V)
print("\nReconstructed matrix:")
print(reconstructed_A)

print("\nReconstructed matrixis equals to A: ", tol >= np.linalg.norm(A-reconstructed_A))


Reconstructed matrix:
[[0.37454012 0.55336647 0.66655448 0.71555056]
 [0.55336647 0.15599452 0.38307809 0.53925763]
 [0.66655448 0.38307809 0.02058449 0.57586741]
 [0.71555056 0.53925763 0.57586741 0.18340451]]

Reconstructed matrixis equals to A:  False


(iii) the eigenvector associated to the biggest eigenvalue of $A$ is the first column of the output matrix $V$.

In [10]:
# Check if eigenvector associated with the biggest eigenvalue of A is the first column of V
eigenvals, eigenvectors = np.linalg.eigh(A)

biggest_eigvec = eigenvectors[:, -1]
first_col_V = V[:, 0]
print("\nEigenvector associated with the biggest eigenvalue of A:")
print(biggest_eigvec)
print("\nFirst column of V:")
print(first_col_V)


Eigenvector associated with the biggest eigenvalue of A:
[-0.58338689+0.j         -0.43576874-0.03078383j -0.44115672-0.06937861j
 -0.51618976-0.05415894j]

First column of V:
[-0.58375278 -0.43543757 -0.44681529 -0.5195988 ]
